In [ ]:
# Uncomment the rows below to install some packages for xlsx and ods files
# %pip install openpyxl
# %pip install odfpy

Handover for Jenny:
The spreadsheet for the footfall data has two different sheets that need to be cleaned. (see note below on which is which). I have started on cleaning the larger of the sheets df_s1/2a. This has gone ok. The difficult thing is both sheets have different 'shapes'.  2a lists months vertically, whereas 1a lists them horizontally. My plan is to get each sheet as clean as I can before the weekend so perhaps together we can forge a plan for the reshaping? NOTE. there is an [r] in the col names AND within number rows on df_s2. df_s1 is clean. The col names are different between the sheets. pandas.merge_asof may be of help to have an almost match.. not looked into it enough. df_s1 uses 'x' to represent missing data, df_s2 uses '0'

JT 22-Nov
* Hello! did some more cleaning: changing NaNs to 0 for s1; standardising Museum names in both (removing bits in brackets, making title case); removing [r] bits in s2.
* I see s1's years are in financial year so that will make things tricky.. can't figure out an easy way to do that yet.. frustrating theyre so differently formatted haha!

JH 25 Nov
I've cleaned the financial year to calendar year in s1. S2 needs reformating. If you look at the bottom i've started. I think you can use .melt() and then .pivot_table() to rearrange it. I think then .map() to rearrange the rows.

JT 26-Nov
* Cleaned rest of combined_df to make NaN's 0 (as there was no data from jul onwards in 2025)
* Updated museum names to remove the year 'total'
* Started some analyses/ visualisations (line graphs, heatmaps)

In [ ]:
import pandas as pd

There are two different footfall sheets that are for monthy visits. df_s1 is the data 2004 to 2018. df_s2 is 2019 to 2025

In [ ]:
df_s1 = pd.read_excel('../data/Monthly_and_Quarterly_Visits_to_DCMS-Sponsored_Museums_and_Galleries_-_to_June_2025_data_tables.ods', sheet_name = '2a')
df_s2 = pd.read_excel('../data/Monthly_and_Quarterly_Visits_to_DCMS-Sponsored_Museums_and_Galleries_-_to_June_2025_data_tables.ods', sheet_name = '1a')

Below are variables I've used to save conditions

In [ ]:
months = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']
museum_group = 'Museum group'

Review the columns

In [ ]:
df_s1.columns

In [ ]:
df_s2.columns

In [ ]:
# The header is not the first row!  This adjusts the row and ignores the notes at the top. can now use col names
df_s1.columns = df_s1.iloc[2]
df_s1 = df_s1[2:]
df_s1.columns

In [ ]:
#removes any row that isn't a month
only_months = df_s1.iloc[:, 2].isin(months)
df_s1 = df_s1[only_months]

In [ ]:
# JT: fill NaNs with 0
df_s1.replace('x', pd.NA, inplace=True)
year_columns = df_s1.columns[3:]
df_s1[year_columns] = df_s1[year_columns].apply(pd.to_numeric, errors='coerce').fillna(0) # if values cant be converted to number, it becomes NaN
df_s1[year_columns] = df_s1[year_columns].astype(int) # when columns converted to numeric, defaults to float > convert back to int
df_s1.head()

In [ ]:
# JT: museum name col name changed to title case
df_s1.columns = df_s1.columns.str.strip() 
df_s1['Museum Name'] = df_s1['Museum Name'].str.title()
df_s1.head()

In [ ]:
# JT: remove bracketed bits in museum name 
df_s1['Museum Name'] = df_s1['Museum Name'].str.replace(r"[\[\(].*?[\]\)]", "", regex=True)
df_s1.head()

In [ ]:
# I thought would be easier to just look at the grouped data? We can change this. Note this also removes the 'total' of the footfall.
#My rational for this is then the set is clean for us to do our own total and won't get in the way of calculation of the mean etc.
# JT: yes this makes sense to me! means there's no duplication 
group_only = df_s1['Coverage'] == museum_group
df_s1 = df_s1[group_only]

In [ ]:
#drop the 2nd col. (col, axis=0 is row, 1 is col, inplace)
df_s1.drop('Coverage', axis=1, inplace=True)

In [ ]:
df_s1.head(10)

In [ ]:
#saving df_s1 as an csv
df_s1.to_csv('../data/df_s1.csv', index = False)

Now cleaning df_s2

In [ ]:
df_s2.columns = df_s2.iloc[2]
df_s2 = df_s2[2:]
df_s2.columns

In [ ]:
group_only = df_s2['Coverage'] == museum_group
df_s2 = df_s2[group_only]

In [ ]:
df_s2.drop('Coverage', axis=1, inplace=True)

In [ ]:
df_s2.drop('Museum Group', axis=1, inplace=True)

In [ ]:
df_s2.drop('Notes', axis=1, inplace=True)
df_s2.head()

In [ ]:
# JT: make title case and remove bracketed bits in museum name 
df_s2.columns = df_s2.columns.str.strip()  
df_s2['Museum Name'] = df_s2['Museum Name'].str.title()
df_s2['Museum Name'] = df_s2['Museum Name'].str.replace(r"[\[\(].*?[\]\)]", "", regex=True)
df_s2.head()

In [ ]:
# JT: remove [r] in cell
year_columns = df_s2.columns[2:]
df_s2[year_columns] = df_s2[year_columns].replace(r"[\[\(].*?[\]\)]", "", regex=True)
df_s2.head()

In [ ]:
# JT: remove [r] in column headings
df_s2.columns = df_s2.columns.str.replace(r"[\[\(].*?[\]\)]", "", regex=True)
df_s2.head()

In [ ]:
#saving df_s2 as an csv
df_s2.to_csv('../data/df_s2.csv', index=False)

In [ ]:
#JH: part below is really messy so I will clean this later! my workings through to rearrnage for the calendar years from financial year

In [ ]:
df_s1.head()

In [ ]:
#JH: gets the museum list for filtering later
museum_list = df_s1['Museum Name'].unique()
print(museum_list)

In [ ]:
#might delete this as was first attempt at the slicing
def get_slices(museum_list, df):
    col_list = []
    fin_year_list = []
    for year_col in df.columns[2:]:
        fin_year_list.append(year_col)
        year_list = year_col.split('/')
        start_year = int(year_list[0])

        apr_to_dec = df.iloc[0:9][year_col]
        jan_to_mar = df.iloc[9:12][year_col]

        col_list.append(start_year)

    return col_list, apr_to_dec, jan_to_mar, fin_year_list

        

        #print(year_list, start_year, col_list)

        #set start_year as the col names
                          

In [ ]:
#JH: not sure if need this (might delete)
def new_cols(col_list, df):
    for month in col_list:
        df[month] = 0
    return df

In [ ]:
#JH: this works!
def rearrange_footfall(col_list, fin_year_list, df, slice_1, slice_2, museum_list):
    for museum in museum_list:
        m_rows = df[df['Museum Name'] == museum]
        print(m_rows)
        for fin_year in fin_year_list:
            year_list = fin_year.split('/')
            start_year = int(year_list[0])
            apr_to_dec = m_rows[fin_year].iloc[0:9]
            jan_to_mar = m_rows[fin_year].iloc[9:12]
            
            df.loc[m_rows.index[0:9], start_year] = apr_to_dec.values
            df.loc[m_rows.index[9:12], start_year + 1] = jan_to_mar.values
          
    return df, m_rows

In [ ]:
cal_df = df_s1.copy()


In [ ]:
col_list, apr_to_dec, jan_to_mar, fin_year_list = get_slices(museum_list, cal_df)

In [ ]:
print(fin_year_list)

In [ ]:
cal_df = new_cols(col_list, cal_df)

In [ ]:
yearly_df, m_rows = rearrange_footfall(col_list, fin_year_list, cal_df, jan_to_mar, apr_to_dec, museum_list)

In [ ]:
#this creates the order for the months to now rearrange
month_map = {'January': 1, 'February': 2, 'March': 3, 'April': 4, 'May': 5, 'June': 6, 'July': 7, 'August': 8, 'September': 9, 'October': 10, 'November': 11, 'December': 12}

In [ ]:
yearly_df['Month'] = yearly_df['Month'].map(month_map)
yearly_df = yearly_df.sort_values(['Museum Name', 'Month'])

In [ ]:
yearly_df = yearly_df.drop(columns=fin_year_list)

In [ ]:
yearly_df.to_csv('../data/df_s1_calendar_year.csv', index=False)

In [ ]:
#df_s2.drop('Month', axis=1, inplace=True)

In [ ]:
# month_num_list = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
# no_of_museums = len(museum_list)

# month_values = month_num_list * no_of_museums

In [ ]:
#df_s2.insert(1, 'Month', month_values)

In [ ]:
df_s2 = pd.read_csv('../data/df_s2.csv')

In [ ]:
year_list = [2019, 2020, 2021, 2022, 2023, 2024, 2025]

In [ ]:
df_s2 = df_s2.melt(id_vars='Museum Name', var_name='Month', value_name='Footfall')

In [ ]:
df_s2['Month'] = df_s2['Month'].str.strip()

In [ ]:
df_s2[['Month', 'Year']] = df_s2['Month'].str.split(' ', expand=True)

In [ ]:
df_s2 = df_s2.pivot(index=['Museum Name', 'Month'], columns='Year', values='Footfall').reset_index()

In [ ]:
df_s2['Month'] = df_s2['Month'].map(month_map)
df_s2 = df_s2.sort_values(['Museum Name', 'Month'])

In [ ]:
df_s2 = df_s2.reset_index(drop=True)

In [ ]:
df_s2.columns.name = None

In [ ]:
df_s2.head(24)

In [ ]:
yearly_df.reset_index()

In [ ]:
yearly_df = yearly_df.reset_index(drop=True)

In [ ]:
yearly_df.columns.name = None

In [ ]:
print(yearly_df)


In [ ]:
combined_df = yearly_df.merge(df_s2, on=['Museum Name', 'Month'])

In [ ]:
print(combined_df.iloc[:, 17])

In [ ]:
combined_df = combined_df.drop(columns=combined_df.columns[17])

# updated final NaN cells to make 0, ensuring all are int
year_columns = combined_df.columns[2:]
combined_df[year_columns] = combined_df[year_columns].apply(pd.to_numeric, errors='coerce').fillna(0) # if values cant be converted to number, it becomes NaN
combined_df[year_columns] = combined_df[year_columns].astype(int) # when columns converted to numeric, defaults to float > convert back to int

In [ ]:
combined_df['Museum Name'].unique()
combined_df['Museum Name'] = combined_df['Museum Name'].str.replace(' Total', '') # removing 'total' in museum name
combined_df['Museum Name'] = combined_df['Museum Name'].str.strip() # removing trailing spaces
combined_df.head(24)

In [ ]:
combined_df.to_csv('../data/footfall_df.csv', index=False)

# Starting some analysis

In [ ]:
# grouping by museum, finding total visits and ordering by descending total visits 
df_no_month = combined_df.drop(columns=["Month"])
annual_df = df_no_month.groupby("Museum Name").sum()
annual_df["Total Visits"] = annual_df.sum(axis=1)
annual_df = annual_df.sort_values("Total Visits", ascending=False)
annual_df

In [ ]:
import matplotlib.pyplot as plt

museums = ['Tate','British Museum','Natural History Museum','National Gallery','V&A','Royal Armouries', 'Wallace Collection', 'National Coal Mining Museum', 'Museum Of The Home']

plt.figure(figsize=(15,10)) # sets width and height 

years = [col for col in annual_df.columns if col != 'Total Visits']

for museum in museums:
    plt.plot(years, annual_df.loc[museum, years], marker='o', linewidth=1, label=museum)

plt.xlabel("Year")
plt.ylabel("Visitors (millions)")
plt.title("Annual Visitors Numbers For All Museums")
plt.xticks(rotation=45)
plt.legend(title="Museum", loc='upper left') 
plt.grid(True)
plt.show()

In [ ]:
growth = annual_df.drop(columns=["Total Visits"]).pct_change(axis=1) * 100
growth = growth.round(2)
# print(growth)

plt.figure(figsize=(14,10))
growth.T.plot(kind='bar', width=0.6)

plt.xlabel("Year")
plt.ylabel("% Growth YoY")
plt.title("Year on Year Growth")
plt.xticks(rotation=45)
plt.legend(title="Museum", bbox_to_anchor=(1.05, 1), loc='upper left') 
plt.grid(axis='y')
plt.show()

In [ ]:
# !pip install seaborn
# !{sys.executable} -m pip install seaborn
import sys
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

df_new = annual_df.drop(columns=["Total Visits"]).copy()

df_text = df_new.applymap(lambda x: f"{x/1_000_000:.2f}")

cmap = sns.cm.rocket_r # reverses heatmap colours

plt.figure(figsize=(14,5))
sns.heatmap(df_new, annot=df_text, fmt="", annot_kws={"size": 8}, cmap=cmap)

plt.xlabel("Year")
plt.ylabel("Museum")
plt.title("Annual Visitors Numbers For All Museums (Millions)")
plt.xticks(rotation=45)
plt.show()

In [ ]:
ff_df = pd.read_csv('../data/footfall_df.csv')

In [ ]:
#turns all values into ints
ff_df.iloc[:, 2].astype(int)
#checks it worked ... but month is now an int...? 
ff_df.dtypes


In [ ]:
ff_df.iloc[:, 2:].boxplot()
